# Загружаем необходимые библиотеки

In [ ]:
# Загрузим необходимые библиотеки
import ipywidgets as widgets
from msticpy.vis.timeline import display_timeline
from msticpy.nbtools import nbwidgets
from msticpy.data.data_providers import QueryProvider

import os
import dotenv
import pandas as pd

## Выбираем провайдер для запроса событий из MP SIEM

In [ ]:
events_api = QueryProvider("MPSIEMEvents", debug=True)

In [ ]:
# функция обработки для MSTICPy
def dataframe_process(df):
    events = df.astype(str)
    events['TimeCreated'] = pd.to_datetime(events['time'])
    events['Computer'] = events['event_src.host']
    events = events[events['msgid'].apply(lambda x: str(x).isdigit())]
    events['EventID'] = events['msgid'].apply(lambda x: int(x))
    events['TimeGenerated'] = pd.to_datetime(events['time'])
    events['NewProcessName'] = events['object.name']
    events['NewProcessId'] = events['object.id']
    events['ParentProcessName'] = events['datafield4']
    events['ProcessId'] = events['datafield2']
    events['SubjectLogonId'] = events['datafield1']
    events['TargetLogonId'] = events['datafield7']
    events['CommandLine'] = events['datafield5']
    events['cmd'] = events['datafield5']
    events['ParentCommandLine'] = events['datafield9']
    events['SubjectUserName'] = events['subject.name']
    events['SubjectUserSid'] = events['subject.id'] 
    events['TenantId'] = ""
    return events

## Подключаемся к серверу и проходим аутентификацию

In [ ]:
dotenv.load_dotenv('../.env', override=True)
SIEM_HOST = os.getenv("SIEM_HOST")
SIEM_USERNAME = os.getenv("SIEM_USERNAME")
SIEM_PASSWORD = os.getenv("SIEM_PASSWORD")
events_api.connect(f"host={SIEM_HOST};username={SIEM_USERNAME};password={SIEM_PASSWORD}")

## Отрисовываем виджет выбора интервала времени

In [ ]:
q_times = nbwidgets.QueryTime(units='days', max_before=30, before=3, max_after=0)
q_times.display()

## Получаем события по фильтру для выбранного диапазона времени

In [ ]:
wmi_process_events = events_api.exec_query(query="msgid=1 and body contains 'wmiprvse'", time_start=q_times.start,time_end=q_times.end)

In [ ]:
wmi_process_events = dataframe_process(wmi_process_events)

## Рисуем таймлайн возникновения событий

In [ ]:
WIDGET_DEFAULTS = {'layout': widgets.Layout(width='95%'),
                   'style': {'description_width': 'initial'}}

display_timeline(wmi_process_events,
               group_by="EventID",
               source_columns=['Computer','ParentCommandLine','CommandLine'],
               legend="inline")

## Запрашиваем дочерние процессы

In [ ]:
spawn_events = events_api.exec_query(query="(msgid = 4688 or msgid = 1) and (object.process.parent.name = 'wmiprvse.exe')", time_start=q_times.start,time_end=q_times.end)

In [ ]:
spawn_events = dataframe_process(spawn_events)

## Строим дерево процессов

In [ ]:
full_tree = spawn_events[["Computer", "SubjectLogonId", "NewProcessId", "NewProcessName", "ProcessId", "ParentProcessName", "TimeCreated", "EventID","TimeGenerated",'TargetLogonId','CommandLine','SubjectUserName']].mp_plot.process_tree()

## Строим граф

In [ ]:
spawn_events.mp_plot.network(
    source_col="SubjectUserName",
    target_col='ProcessId',
    source_attrs=[ "SubjectLogonId"],
    target_attrs=["NewProcessName", "ParentProcessName", "CommandLine"],
    edge_attrs=["TimeGenerated"],
)